As always, let's start with a few imports:

In [ ]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.metrics import MeanSquaredError

import os


Next, let's see what kind if data we have:

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Read the data
We set some parameters and write a function to read the data

In [ ]:
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

RootDir = "/kaggle/input/cryptocurrencypricehistory"
History = 60

def read_data ():
    coin_no = 0
    for name in os.listdir(RootDir):
        coin_no += 1
    
    max_length, min_length = 0, 1000000
    for name in os.listdir(RootDir):
        df = pd.read_csv(RootDir + "/" + name, parse_dates=['Date'])
        length = df.shape[0]
        if max_length < length:
            max_length = length
        if min_length > length:
            min_length = length
    
    data = np.zeros ((coin_no, max_length))
    lengths = np.zeros(coin_no, dtype = int)
    i = 0
    for name in os.listdir(RootDir):
        short_name = name[5:-4]
        df = pd.read_csv(RootDir + "/" + name, parse_dates=['Date'])
        length = df.shape[0]
        lengths[i] = length
        print (i, short_name, length)
        data[i, 0:length] = df['Close'].values  # We only keep the closing price as a sequence!
        i += 1
    
    return coin_no, lengths, data

coin_no, lengths, data = read_data ()
print ("Got", coin_no, "coins.")

# Scaling and splitting:
We need to scale the data:

In [ ]:
def scale_data (data, lengths):
    coin_no = data.shape[0]
    shift = np.zeros (coin_no)
    factor = np.zeros (coin_no)
    for i in range (coin_no):
        max_val = data[i,:lengths[i]].max()
        min_val = data[i, :lengths[i]].min()
        shift[i] = min_val
        factor[i] = max_val - min_val
        data[i,0:lengths[i]] = (data[i,0:lengths[i]]-shift[i])/factor[i]
    return (shift, factor)
    
shift, factor = scale_data (data, lengths)

Coins 0-18 will be our training set. We'll use the next 4 coins for validation, and we'll use the last coin (XRP) for testing 

In [ ]:
def create_sequences (data, lengths, start, end):
    x = []
    y = []
    for i in range (start, end):   # Go only over the specified coins
        for j in range(History, lengths[i]):
            x.append(data[i, j-History:j])
            y.append(data[i, j])
    return np.array(x)[:, :, np.newaxis], np.array(y)

x_train, y_train = create_sequences(data, lengths, 0, 18)
print ("Got", y_train.shape[0], "training sequenes.")
x_val, y_val = create_sequences(data, lengths, 18, 22)
print ("Got", y_val.shape[0], "validation sequenes.")
x_test, y_test = create_sequences(data, lengths, 22, 23)
print ("Got", y_test.shape[0], "test sequenes.")

# LSTM model
We build a model based on the LSTM architecture to model the data

In [ ]:
def build_lstm ():
    # Build an LSTM model
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (History, 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=[MeanSquaredError()])
    model.summary()
    
    return model

model = build_lstm()

Now, let's train the model we just built using the training set:

In [ ]:
hist = model.fit(x_train, y_train, validation_data = (x_val, y_val), 
              batch_size=32, epochs=5) 

Let's write a function to show the training stats:

In [ ]:
def show_stats (hist):
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title("Model loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend(["Training loss","Validation loss"])
    plt.show()

show_stats (hist)

A bit of overfitting at the end, but it doesn't seem to be too bad. In retrospect, one or two epochs should be enough.

Now that the model is trained, we can use it to predict the test data:

In [ ]:
predictions = model.predict(x_test)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print ("Root mean square error on test data:", rmse)    
plt.plot(predictions*factor[22] + shift[22])
plt.plot(y_test*factor[22] + shift[22])    
plt.legend(["Predictions","Real data"])
plt.show()

Seems like the predictions are pretty accurate!